In [ ]:
%run '/home/christianl/Zhang-Lab/Zhang Lab Code/Boilerplate/Fig_config_utilities.py'

In [ ]:
# Corrected and enhanced Bland-Altman plot script

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

def bland_altman_plot(data1, data2, ax, model1_name, model2_name, confidence_interval=1.96):
    """
    Helper function to create a single Bland-Altman plot on given axis.
    
    Parameters
    ----------
    data1, data2 : array-like
        Predictions from two models (should be same shape)
    ax : matplotlib axis
        Axis to plot on
    model1_name, model2_name : str
        Names of models being compared
    confidence_interval : float
        Z-score for confidence interval (default 1.96 for 95% CI)
    
    Returns
    -------
    dict : Statistics including mean difference, SD, and limits
    """
    # Convert to numpy arrays and flatten
    data1 = np.asarray(data1).ravel()
    data2 = np.asarray(data2).ravel()
    
    # Check shapes match
    if data1.shape != data2.shape:
        raise ValueError(f"Data shapes don't match: {data1.shape} vs {data2.shape}")
    
    # Calculate mean and difference
    mean = (data1 + data2) / 2  # This is correct!
    diff = data1 - data2
    
    # Statistics
    md = np.mean(diff)  # Mean difference
    sd = np.std(diff, ddof=1)  # Standard deviation (use ddof=1 for sample SD)
    
    # Limits of agreement (typically ±1.96 SD for 95% CI)
    ci_low = md - confidence_interval * sd
    ci_high = md + confidence_interval * sd
    
    # Scatter plot (subsample if too many points)
    n_points = len(mean)
    if n_points > 50000:
        # Subsample for plotting performance
        sample_size = 50000
        indices = np.random.choice(n_points, sample_size, replace=False)
        mean_plot = mean[indices]
        diff_plot = diff[indices]
        alpha = 0.3
    else:
        mean_plot = mean
        diff_plot = diff
        alpha = 0.5
    
    ax.scatter(mean_plot, diff_plot, alpha=alpha, s=20, 
               color='steelblue', edgecolors='none')
    
    # Mean difference line
    ax.axhline(md, color='red', linestyle='-', lw=2.5, 
               label=f'Mean diff: {md:.4f}')
    
    # Zero line (perfect agreement)
    ax.axhline(0, color='black', linestyle='--', lw=1.5, alpha=0.5,
               label='Perfect agreement')
    
    # Limits of agreement (confidence interval lines)
    ax.axhline(ci_high, color='gray', linestyle='--', lw=2, alpha=0.7,
               label=f'±{confidence_interval}σ limits')
    ax.axhline(ci_low, color='gray', linestyle='--', lw=2, alpha=0.7)
    
    # Calculate percentage of points within limits
    within_limits = np.sum((diff >= ci_low) & (diff <= ci_high))
    pct_within = 100 * within_limits / len(diff)
    
    # Add text annotations for limits
    x_pos = mean.max() * 0.98
    ax.text(x_pos, ci_high, f'+{confidence_interval}σ: {ci_high:.4f}',
           fontsize=9, verticalalignment='bottom', horizontalalignment='right',
           bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    ax.text(x_pos, ci_low, f'-{confidence_interval}σ: {ci_low:.4f}',
           fontsize=9, verticalalignment='top', horizontalalignment='right',
           bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    # Statistics box
    textstr = (f'Mean diff: {md:.4f}\n'
               f'SD: {sd:.4f}\n'
               f'Within limits: {pct_within:.1f}%')
    ax.text(0.02, 0.98, textstr, transform=ax.transAxes,
           fontsize=9, verticalalignment='top',
           bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # Check for proportional bias (correlation between mean and difference)
    corr, p_value = pearsonr(mean, diff)
    if abs(corr) > 0.1 and p_value < 0.05:
        # Add trend line to show proportional bias
        z = np.polyfit(mean, diff, 1)
        p = np.poly1d(z)
        x_trend = np.linspace(mean.min(), mean.max(), 100)
        y_trend = p(x_trend)
        ax.plot(x_trend, y_trend, 'r--', lw=1.5, alpha=0.5,
               label=f'Trend (r={corr:.3f})')
        
        # Add warning about proportional bias
        if abs(corr) > 0.3:
            ax.text(0.98, 0.02, '⚠ Proportional bias detected', 
                   transform=ax.transAxes,
                   fontsize=9, verticalalignment='bottom', 
                   horizontalalignment='right',
                   bbox=dict(boxstyle='round', facecolor='orange', alpha=0.8))
    
    # Labels and formatting
    ax.set_xlabel(f'Mean of {model1_name} & {model2_name}', 
                  fontsize=11, fontweight='bold')
    ax.set_ylabel(f'Difference ({model1_name} - {model2_name})', 
                  fontsize=11, fontweight='bold')
    ax.set_title(f'{model1_name} vs {model2_name}', 
                 fontsize=12, fontweight='bold')
    ax.legend(loc='upper left', fontsize=8)
    ax.grid(True, alpha=0.3)
    
    # Return statistics
    return {
        'mean_diff': md,
        'sd': sd,
        'ci_low': ci_low,
        'ci_high': ci_high,
        'pct_within_limits': pct_within,
        'proportional_bias_r': corr,
        'proportional_bias_p': p_value
    }


def figure_3_bland_altman(predictions_dict, 
                         output_path='/home/christianl/Zhang-Lab/Zhang Lab Code/Figures/figure_3_bland_altman.png'):
    """
    Generate Bland-Altman plots for pairwise model comparisons.
    
    Parameters
    ----------
    predictions_dict : dict
        Dictionary of predictions by model
        Example: {'MLR': pred_mlr, 'XGBRFRegressor': pred_xgbrf, 'RNN': pred_rnn}
    output_path : str
        Path to save figure
    
    Returns
    -------
    dict : Statistics for each comparison
    """
    set_publication_style()
    model_names = list(predictions_dict.keys())
    n_models = len(model_names)
    
    if n_models < 2:
        raise ValueError("Need at least 2 models for comparison")
    
    # all pairwise comparisons
    from itertools import combinations
    comparisons = list(combinations(model_names, 2))
    n_comparisons = len(comparisons)
    
    print(f"\nGenerating {n_comparisons} Bland-Altman plots for pairwise comparisons...")
    
    # subplot layout
    if n_comparisons == 1:
        fig, axes = plt.subplots(1, 1, figsize=FIGSIZE_SINGLE)
        axes = [axes]
    elif n_comparisons == 2:
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    elif n_comparisons == 3:
        fig, axes = plt.subplots(1, 3, figsize=FIGSIZE_TRIPLE)
    elif n_comparisons == 4:
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        axes = axes.ravel()
    elif n_comparisons <= 6:
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        axes = axes.ravel()
    else:
        # For more than 6 comparisons, create a grid
        n_cols = 3
        n_rows = (n_comparisons + n_cols - 1) // n_cols
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5*n_rows))
        axes = axes.ravel()
    
    # generate plots
    statistics = {}
    
    for idx, (model1, model2) in enumerate(comparisons):
        ax = axes[idx]
        print(f"  Comparing {model1} vs {model2}...")
        
        stats = bland_altman_plot(
            predictions_dict[model1],
            predictions_dict[model2],
            ax, model1, model2
        )
        
        statistics[f'{model1}_vs_{model2}'] = stats
    
    # hide unused subplots
    for idx in range(n_comparisons, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=DPI, bbox_inches='tight')
    print(f"\nFigure 3 saved to {output_path}")
    plt.show()
    
    # summary stats
    print("\n" + "="*80)
    print("BLAND-ALTMAN COMPARISON SUMMARY")
    print("="*80)
    
    for comparison, stats in statistics.items():
        print(f"\n{comparison}:")
        print(f"  Mean difference: {stats['mean_diff']:8.4f}")
        print(f"  SD of difference: {stats['sd']:7.4f}")
        print(f"  95% limits: [{stats['ci_low']:7.4f}, {stats['ci_high']:7.4f}]")
        print(f"  Within limits: {stats['pct_within_limits']:5.1f}%")
        
        if abs(stats['proportional_bias_r']) > 0.1 and stats['proportional_bias_p'] < 0.05:
            print(f"  ⚠ Proportional bias: r={stats['proportional_bias_r']:.3f}, p={stats['proportional_bias_p']:.2e}")
        
        # interpretation
        if abs(stats['mean_diff']) < 0.01:
            print("  → Models show excellent agreement (negligible bias)")
        elif abs(stats['mean_diff']) < 0.1:
            print("  → Models show good agreement (minor bias)")
        else:
            print("  → Models show systematic difference")
    
    print("="*80 + "\n")
    
    return statistics


In [ ]:
# setting figure dimensions slightly bigger for more breathing room between subplots 
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

In [ ]:
# MLR vs. XGBRF
bland_altman_plot(mlr_y_pred, xgbrf_y_pred, axes[0], 'MLR', 'XGBRF', confidence_interval=1.96)

In [ ]:
# MLR vs. RNN
bland_altman_plot(mlr_y_pred, rnn_y_pred, axes[1], 'MLR', 'RNN', confidence_interval=1.96)

In [ ]:
# XGBRF vs. RNN
bland_altman_plot(xgbrf_y_pred, rnn_y_pred, axes[2], 'XGBRF', 'RNN', confidence_interval=1.96)

In [ ]:
plt.tight_layout()
plt.show()